In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool


/Users/ali/anaconda3/envs/graph_rag/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(model='gpt-4o',temperature=0)

In [4]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

In [5]:
from langchain.output_parsers import PydanticOutputParser
from typing import Literal
from langchain.memory import ChatMessageHistory


In [6]:
def evalation_generator(questions=[],answers=[]):
    """
    An Experience Evalaution agent that Evalauate Question and Answers
    """

    class Evalution(BaseModel):
        correct: bool = Field(description="Boolean value which tell user response is correct or not")
        score: int = Field(description="Score of the user response how close it is")
        feedback: str = Field(description="Feedback for the user response or suggestion what should be response")
    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a professional exam evaluation agent."
            "You will be given scenario, data , question and its answer."
            "You will  evaluate the answer and provide feedback."
            "You will rate user response from 1 to 10."
            "You will mark  the response as correct or incorrect."
            "You will return a json with keys : 'correct', 'score', 'feedback' "
            "Do not provide uncessery feedback"
        ),
        ("human", "Based on this data {data} and {scenario} also you own knowledge, evaluate user response {answer} to this question {question}"),
    ]
    )
    parser = PydanticOutputParser(pydantic_object=Evalution) 
    chain = prompt | llm | parser
    answers = answers[:len(questions)]
    pairs_list = [{"scenario":'Maths Arithematic','data':'It is about basic maths','answer':answers[_],'question':questions[_]} for _ in range(len(questions))]
    ai_msg = chain.batch(pairs_list)

    return ai_msg
    

In [7]:
evalation_generator(questions=['2+2','0/0','3*13'],answers=[4,'0/0 is undefined as we can not calculate it','39'])

[Evalution(correct=True, score=10, feedback='The answer is correct. 2 + 2 equals 4.'),
 Evalution(correct=True, score=10, feedback='The response is correct. Division by zero is undefined, and 0/0 is a classic example of an indeterminate form.'),
 Evalution(correct=True, score=10, feedback='Correct answer.')]

In [19]:
class ExamBotStateMachine:
    def __init__(self) -> None:
        self.state = {}
        self.construct_state()

    def construct_state(self):
        self.state  = {
            'topic':'',
            'data': '',
            'scenario':'',
            'questions':[],
            'answers': [],
            'chat_hisotry': ChatMessageHistory(),
            'evaluation':[]
        }

    def forward(self,user_input):
        if len(self.state['answers']) >= 3 and len(self.state['answers']) >= 3:
            self.evaluation_generator()
            return 'The END'
        resp = ''
        self.state['chat_hisotry'].add_user_message(user_input)
        if self.state['scenario'] == '' or self.state['scenario'] == None:
            self.get_data()
            self.scenario_generator()
            resp = self.state['scenario']
        elif len(self.state['questions'])  == 0:
            resp = self.question_generator()
        else:
            next_node = self.router_node(user_input)
            print(user_input,next_node)
            if next_node == 'QUESTION_NODE':
                self.state['answers'].append(user_input)
                resp = self.question_generator()
            elif  next_node == 'CLARIFICATION_NODE':
                last_question = self.state['questions'][-1]
                resp = self.clarification_node(last_question,user_input)
            else:
                resp = self.general_conversation_agent()
        
        self.state['chat_hisotry'].add_ai_message(resp)
        return resp
    def add_topic(self,topic):
        self.state['topic'] = topic
    
    def get_data(self):
        """
        A function to generate data;
        """
        prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a highly specialized data generator agent. You receive a topic from the user and generate a comprehensive, informative description of the topic, "
                "ensuring accuracy and depth. The description should be fact-based, logically structured, and written in a professional tone."
            ),
            ("human", "Please generate a detailed description for the topic: {topic}.")
        ]
         )

        chain = prompt | llm
        ai_msg = chain.invoke({"topic":self.state['topic']})
        self.state['data'] = ai_msg.content
        return ai_msg.content
    

    def scenario_generator(self):
        """
        An Experience Scenario maker that design scenario for interview.
        """
        class Scenario(BaseModel):
            scenario: str = Field(description="The generated scenario for paticular topic")
        prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a professional examiner and interviewer tasked with generating a realistic and concise scenario for an interview or exam. "
                "The scenario should provide a real-world situation based on the given topic and data. It should be clear, contextual, and no more than 3 sentences. "
                "The scenario will be returned in JSON format with the key 'scenario'."
            ),
            ("human", "Please generate a real-world scenario for the topic {topic} using the provided data: {data}. Return the scenario in JSON format.")
        ]
         )

        parser = PydanticOutputParser(pydantic_object=Scenario) 
        chain = prompt | llm | parser
        ai_msg = chain.invoke({"topic":self.state['topic'],'data':self.state['data']})
        self.state['scenario'] = ai_msg.scenario
        return ai_msg.scenario + "\n If you want to start the exam please respond with start exam."

    
    def question_generator(self):
        """
        You are an examiner and you need evaluate a candiate 's knowledge of a particular subject.
        You will ask questions based on given scenario, and data. 
        Also you will have access to previous asked questions. There might be previous asked questions.
        You will not answer question your self. You also will not help user in question.
        """
        
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are a professional examiner conducting an evaluation on a specific topic. You will generate a series of questions based on the given scenario and data. "
                    "The questions should progressively assess the depth of the candidate's understanding, ranging from basic to advanced levels. "
                    "You will generate only one question at a time. Not more then one questions"
                    "Ensure that the questions do not provide answers or hints. Return only the question text without any preamble or additional information."
                ),
                ("human", "Generate a question for the exam on the topic {topic}, considering the provided scenario: {data} and previous questions: {previous_questions}.")
            ]
        )

        chain = prompt | llm 
        ai_msg = chain.invoke({"topic":self.state['topic'],'data':self.state['data'],
                               'previous_questions':self.state['questions']})
        self.state['questions'].append(ai_msg.content)
        return ai_msg.content
        
    def router_node(self,user_input):
        """
        You are a router node that need to decide  which path to take.
        """
        
        # from typing import Literal
        class PathObj(BaseModel):
            next_node: Literal['QUESTION_NODE', 'LLM_NODE', 'CLARIFICATION_NODE'] = Field(
                description="The next step the state machine should follow based on the user's input. "
                            "Valid options are: 'QUESTION_NODE' if user answer to this question and now we give him next question, 'CLARIFICATION_NODE' if user asking for help in question, "
                            "or 'LLM_NODE' for handling general conversation.",
                example="QUESTION_NODE"
            )

        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are a routing node responsible for determining the next logical step based on user input. "
                    "You will return next_node as QUESTION_NODE if User answered to given question or It is not asking for clarification or general chat. "
                    "If user say I don't know or I quit go to next question"
                    "If user is asking for clarification  or general chat, return CLARIFICATION_NODE or LLM_NODE respectively."
                    "You MUST return a valid JSON object with a key 'next_node' and value being one of ['QUESTION_NODE', 'LLM_NODE', 'CLARIFICATION_NODE']. "
                    "Always ensure the output is in the format"
                ),
                ("human", "Given the user input: {input} on  previous question: {question}, which of the following nodes should be selected: ['QUESTION_NODE', 'LLM_NODE', 'CLARIFICATION_NODE']?")
            ]
        )

        parser = PydanticOutputParser(pydantic_object=PathObj)
        chain = prompt | llm | parser

        # Safeguard against invalid output by wrapping in a try-except block
        try:
            given_question = self.state['questions'][-1] if len(self.state['questions']) > 0 else []
            ai_msg = chain.invoke({"input": user_input, 'question': given_question})
            return ai_msg.next_node
        except Exception as e:
            print(f"Error in parsing router node output: {e}")
            return 'LLM_NODE'  # Default fallback if parsing fails
    
    def clarification_node(self,restate_input,user_response):
        """
        """
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are a professional examiner who provides clarifications when candidates do not understand the question. "
                    "You will rephrase or restate the question in simpler, more understandable language without providing any hints or answers."
                ),
                ("human", "Rephrase the question: {restate_input}. The user responded with: {user_response}.")
            ]
        )

        chain = prompt | llm 
        ai_msg = chain.invoke({"restate_input":restate_input,'user_response':user_response})
        return ai_msg.content
    
    def general_conversation_agent(self):
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are an ExamBot conducting an examination. When engaging in general conversation, guide the user back to the exam. "
                    "You may provide friendly or neutral responses, but avoid answering exam-related questions. "
                    "Ensure the output is returned as a JSON object with the key 'text_msg'."
                ),
                MessagesPlaceholder(variable_name="chat_history")
            ]
        )


        chain = prompt | llm 
        ai_msg = chain.invoke({'chat_history':self.state['chat_history'].messages})
        return ai_msg.content
    
    def evaluation_generator(self):
        """
        An Experience Evaluation agent that evaluates Question and Answers.
        """

        class Evaluation(BaseModel):
            status: str = Field(
                description="Indicates whether the user's answer is correct or incorrect. The value must be either 'correct' or 'incorrect'.",
                example="correct"
            )
            score: int = Field(
                description="A score between 1 and 10 based on how accurate the user's answer is. "
                            "Higher scores indicate closer matches to the expected answer.",
                ge=1,
                le=10,
                example=8
            )
            feedback: str = Field(
                description="Constructive feedback for the user, explaining the strengths and weaknesses of their answer. "
                            "Suggestions for improvement can also be included here.",
                example="Good understanding of the topic, but missed key points regarding budget management."
            )

        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are a professional exam evaluator. Based on the provided scenario, data, question, and user's answer, "
                    "you will assess the quality and correctness of the response. "
                    "You will assign a score from 1 to 10 based on how close the answer is to the correct response, and you will provide constructive feedback. "
                    "Indicate if the answer is 'correct' or 'incorrect'. "
                    "Respond strictly in a valid JSON format with keys: 'status', 'score', and 'feedback'."
                ),
                ("human", "Evaluate the user's response: {answer} to the question: {question}, based on the scenario: {scenario} and data: {data}.")
            ]
        )

        parser = PydanticOutputParser(pydantic_object=Evaluation)

        # Define the LLM chain
        chain = prompt | llm | parser

        questions = self.state['questions']
        answers = self.state['answers']


        questions = questions[:len(answers)]

        pairs_list = [{"scenario": self.state['scenario'], 'data': self.state['data'], 'answer': answers[i], 'question': questions[i]} for i in range(len(questions))]

        try:
            # Run the chain in batch mode for multiple Q&A pairs
            ai_msg = chain.batch(pairs_list)
            self.state['evaluation'] = ai_msg
            return ai_msg

        except Exception as e:
            # Check if JSON parsing failed and handle invalid outputs
            print(f"Error in parsing JSON output: {e}")
            return {"error": "Invalid output received from model. Ensure the output adheres to the required JSON format."}


    


In [20]:
exam_bot = ExamBotStateMachine()

In [21]:
exam_bot.add_topic('Machine Learning')

In [22]:
result = exam_bot.forward('Hi')

In [23]:
while True:
    if 'END' in result:
        break
    user_resp = input('Please say something: ')
    result = exam_bot.forward(user_resp)
    print(result)

What is the difference between supervised and unsupervised learning in machine learning, and can you provide an example of an algorithm used for each type?
Supervised Learning Data is prelabelled example linear regression, while in Unsuperviseed learning data in not labelled example KNN.  QUESTION_NODE
Explain the concept of overfitting in machine learning and discuss one method to prevent it.
Overfitting mean model is predicting good on training data while not well on test data. QUESTION_NODE
Describe the process of hyperparameter tuning in machine learning and explain why it is important for model performance.
it is important because model optimize well in different condition; like small batch size model will take more time to learn etc . Large LR model will stuck in local minima QUESTION_NODE
Discuss the role of data and features in machine learning and explain how they impact the performance of a model.
The END


In [25]:
exam_bot.state

{'topic': 'Machine Learning',
 'data': 'Machine Learning (ML) is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform tasks without explicit instructions. It involves the use of data to train models, allowing them to make predictions or decisions based on input data. The core idea is to enable machines to learn from experience, adapt to new inputs, and perform tasks that are traditionally considered to require human intelligence.\n\n### Key Concepts in Machine Learning\n\n1. **Data and Features**: Data is the foundation of machine learning. It consists of examples or instances that the model learns from. Features are the individual measurable properties or characteristics of the data. The quality and quantity of data significantly impact the performance of machine learning models.\n\n2. **Algorithms**: Machine learning algorithms are the mathematical engines that drive the learning process. They c

In [13]:
result = exam_bot.forward('Go ahead')

In [14]:
exam_bot.state['questions']

['What are the main types of machine learning algorithms, and how do they differ in terms of the data they are trained on?']

In [294]:
result

'What are the main types of machine learning algorithms, and what are the key differences between supervised, unsupervised, semi-supervised, and reinforcement learning?'

In [15]:
result = exam_bot.forward('Supervised learning data is labelled, unsupervised larning it is not labelled, in reinforcement learning reward is given')

Supervised learning data is labelled, unsupervised larning it is not labelled, in reinforcement learning reward is given QUESTION_NODE


In [16]:
result

"How does the training process in machine learning contribute to the model's ability to make predictions on new, unseen data?"

In [17]:
exam_bot.state['questions']

['What are the main types of machine learning algorithms, and how do they differ in terms of the data they are trained on?',
 "How does the training process in machine learning contribute to the model's ability to make predictions on new, unseen data?"]

In [18]:
result = exam_bot.forward('In the training process weights are updated. Which take part in generalizing on unseen data')

In the training process weights are updated. Which take part in generalizing on unseen data QUESTION_NODE


In [19]:
exam_bot.state['questions']

['What are the main types of machine learning algorithms, and how do they differ in terms of the data they are trained on?',
 "How does the training process in machine learning contribute to the model's ability to make predictions on new, unseen data?",
 'How does reinforcement learning differ from supervised and unsupervised learning in machine learning, and what kind of tasks is reinforcement learning typically used for?']

In [20]:
result =  exam_bot.forward('It is based on agent reward on punishment on differnt action')

It is based on agent reward on punishment on differnt action QUESTION_NODE


In [21]:
result

'What are some key applications of machine learning in different industries, and how does it contribute to advancing technology and society as a whole?'

In [22]:
exam_bot.state

{'topic': 'Machine Learning',
 'data': "Machine learning is a subfield of artificial intelligence (AI) that involves the development of algorithms and statistical models that enable computers to learn from and make predictions or decisions based on data without being explicitly programmed. It is a rapidly evolving and highly interdisciplinary field that combines principles from computer science, mathematics, statistics, and domain-specific knowledge to create intelligent systems that can automatically learn and improve from experience.\n\nAt the core of machine learning is the concept of training a model on a dataset to recognize patterns and relationships within the data. This training process involves providing the model with labeled examples, which are used to adjust the model's parameters and optimize its performance. Once trained, the model can then be used to make predictions or decisions on new, unseen data.\n\nThere are several types of machine learning algorithms, each suited 

In [23]:
result = exam_bot.forward('It can be used for diagnosising like X-Ray MRI using CNN')

In [24]:
exam_bot.state

{'topic': 'Machine Learning',
 'data': "Machine learning is a subfield of artificial intelligence (AI) that involves the development of algorithms and statistical models that enable computers to learn from and make predictions or decisions based on data without being explicitly programmed. It is a rapidly evolving and highly interdisciplinary field that combines principles from computer science, mathematics, statistics, and domain-specific knowledge to create intelligent systems that can automatically learn and improve from experience.\n\nAt the core of machine learning is the concept of training a model on a dataset to recognize patterns and relationships within the data. This training process involves providing the model with labeled examples, which are used to adjust the model's parameters and optimize its performance. Once trained, the model can then be used to make predictions or decisions on new, unseen data.\n\nThere are several types of machine learning algorithms, each suited 

In [25]:
result

'The END'